In [43]:
import numpy as np
import pandas as pd
import sys
from astropy import units as u
from astropy.coordinates import SkyCoord

from astropy.coordinates import GeocentricTrueEcliptic
sys.path.insert(1, 'mm_SPINNY/')
sys.path.insert(2, 'prep/')
from mm_SPINNY.spinny_vector import generate_vector
import prep.latlon_transform as latlon_transform
import mm_priors as prior
import mm_param
import random
import commentjson as json

from astropy.time import Time
import mm_relast
from csv import writer
import latlon_transform
import os
import time
from scipy.stats import chi2
from astroquery.jplhorizons import Horizons
import mpmath as mp
#from func_timeout import func_timeout, FunctionTimedOut


class ReadJson(object):
    def __init__(self, filename):
        print('Read the runprops.txt file')
        self.data = json.load(open(filename))
    def outProps(self):
        return self.data

"""
Inputs:
1)The Parameters dataframe
2) The Observation Dataframe
Outputs:
1) The chi-squared number of the likelihood
"""
# calculates the chi-square for parameters given observations
def mm_chisquare(paramdf, obsdf, runprops, geo_obj_pos, gensynth = False):

    #print('paramdf',paramdf)
    numObj = runprops.get("numobjects")
    verbose = runprops.get("verbose")
    pd.set_option('display.max_columns', None)
    names = []
    for i in range(1,numObj+1):
        names.append('name_'+str(i))
        if not 'name_'+str(i) in paramdf.columns:
            print('The parameter name_' + str(i)+ ' is not found in the parameter dataframe.')
            sys.exit()
        
    
    # use parameters dataframe with one set of parameters and observation times to call SPINNY to get the model dataframe
    # SPINNY returns (full?) state vector of each object (in primaricentric ecliptic J2000 coordinates) at each time
    
    # parameters dataframe "paramdf" is 1 row and many columns
    # N objects, each of which is a point-mass, oblate, or triaxial
       # point-masses: mass, 6 orbital elements
       # oblate: + J2R2, spinc, splan
       # triaxial: + C22R2, spaop, sprate
       # dynamicstoincludeflags='2010'
    # Include Sun or not
    # it has already been "checked" to make sure that all of the columns that are needed are there
    # mass_0, sma_0, ecc_0, ..., mea_0 are the orbital elements of the Sun 
       # at the epoch in primaricentric coordinates
    # when columns are not needed they are typically absent

    # MultiMoon units are km, kg, deg, seconds

    obsdf = obsdf.sort_values(by=['time'])
    #time_arr = np.sort(obsdf['time'].values.flatten())
    #print('OBSDF: ',obsdf)
    time_arr = obsdf['time'].values.flatten()# gets an array of observation times from the obs dataframe

    # Setting times relative to the epoch
    epoch = runprops.get("epoch_SJD")
    time_arr = time_arr - epoch

    # Sorts them into ascending order
#    import logging
    #print('time_Arr', time_arr)
    #print('obsdf', obsdf)

    begin = time.time()
    #print(paramdf)
    #print('param_df:', paramdf)
    try:
        time_arr_sec = time_arr*86400
        #vec_df = func_timeout(5,generate_vector,args=(paramdf, time_arr_sec, runprops))
        #print(paramdf)
        #print('time_arr_sec: ',time_arr_sec)
        vec_df = generate_vector(paramdf, time_arr_sec, runprops)
        
    #except FunctionTimedOut:
    #    print('Spinny took longer than 5 seconds to run 1 walker-step:\n')
    #    return np.inf
#    except Exception as e:
#        logging.exception('')
#        return np.inf
    except Exception as e:
        print('There was an error thrown within spinny:\n', e)
        rows = obsdf.shape[0]
        return -np.inf, np.ones(((numObj-1)*2, rows))*10000
    names_dict = runprops.get("names_dict")
    names=[0 for i in range(numObj)]
    for i in range(0,numObj):
        names[i] = names_dict.get("name_"+str(i+1))
    end = time.time()
    #print(end-begin,' seconds')
    #if end-begin > 2:
    #    print('A step took ', end-begin,' seconds')
    #    print(paramdf)

    # vec_df is a dataframe with len(time_arr) rows and
    # columns are state parameters x nobjects
    # Example: vecdf["X_Pos_"+paramsdf["name_2"]] gets the x position of object 2
    # ecliptic (J2000) coordinates
    # km, kg, rad, s
    # primaricentric 

    name_1 = "X_Pos_"+names[0]
    #print(vec_df)
    if (vec_df[name_1][0] != 0.0):
        print("Not primaricentric like I thought!")
        rows = obsdf.shape[0]
        return -np.inf, np.ones(((numObj-1)*2, rows))*10000
        #print("vec_df[name_1] = ", vec_df)
    
    Model_DeltaLong = np.zeros((numObj-1,len(time_arr)))
    Model_DeltaLat = np.zeros((numObj-1,len(time_arr)))
    if runprops.get('includesun') == 1:
        #print(vec_df)
        vec_df = vec_df.drop(['X_Pos_Sun', 'Y_Pos_Sun', 'Z_Pos_Sun', 'X_Vel_Sun', 'Y_Vel_Sun', 'Z_Vel_Sun'], axis=1)

    positionData = np.zeros((numObj*3,len(time_arr)))
        
    for i in range(0,numObj):
        positionData[3*i] = vec_df["X_Pos_"+names[i]]
        positionData[3*i+1] = vec_df["Y_Pos_"+names[i]]
        positionData[3*i+2] = vec_df["Z_Pos_"+names[i]]

        # tind = index/row number of vec_df corresponding to this time
        
        # for object from 2:N
             # gather relative positions
             # thisdx = vec_df[tind,"X_Pos_"+paramdf["name_"+str(object)] [ - X_Pos_1=0]
             # same for dy and dz

        # Implicitly assume that observer is close to geocenter (within ~0.01 AU)
        
        # obs_to_prim_pos = vectors of observer to primary
        # prim_to_sat__pos = vectors of primary to satellite

    obs_to_prim_pos = [positionData[0]+geo_obj_pos['x'].tolist(),positionData[1]+geo_obj_pos['y'].tolist(),positionData[2]+geo_obj_pos['z'].tolist()]
    
    prim_to_sat_pos = [positionData[1*3],positionData[1*3+1],positionData[1*3+2]]
    Model_DeltaLong, Model_DeltaLat = mm_relast.convert_ecl_rel_pos_to_geo_rel_ast(obs_to_prim_pos, prim_to_sat_pos)
    return obs_to_prim_pos,Model_DeltaLong,Model_DeltaLat


In [44]:
objname = 'Eris'
file = 'Eris_2023-09-14_21.42.55_00_robust_2018_v6'
filename = '../results/'+objname+'/'+file
getData = ReadJson(filename+'/runprops.txt')
runprops = getData.outProps()
obsdf = pd.read_csv(filename+'/'+objname+'_obs_df.csv')
location = '@SEMB-L2'

Read the runprops.txt file


In [77]:
epoch = runprops.get('epoch_SJD')
time_to_L2 = 5.016722408026756/24/60/60

total_model = pd.DataFrame()
total_ra = np.zeros(150*365)
total_dec = np.zeros(150*365)
for i in range(150):
    if i%10 == 0:
        print(i)
    times = np.arange(2455680,2455680+36.5,0.1)
    ourKBO = Horizons(id=objname,location=location,epochs = times)
    ephKBO = ourKBO.ephemerides()['RA','DEC','datetime_jd']
    vecKBO = ourKBO.vectors(aberrations = 'astrometric')['lighttime','x','y','z']

    jdTime= ephKBO['datetime_jd']
    lightTime = vecKBO['lighttime']
    kboTime=jdTime-lightTime

    geo_obj_pos = pd.DataFrame({'kboTIME':kboTime,'x':vecKBO['x']*149597870.7 ,'y':vecKBO['y']*149597870.7 ,'z':vecKBO['z']*149597870.7 })
    geo_obj_pos['time'] = jdTime-time_to_L2
    paramdf = pd.read_csv(filename+'/'+objname+'_init_guess.csv',index_col=0)

    paramdf = paramdf.T
    paramdf['name_1'] = objname
    paramdf['name_2'] = runprops.get('names_dict').get('name_2')
    paramdf['time'] = epoch
    data = np.array(mm_chisquare(paramdf, geo_obj_pos, runprops, geo_obj_pos))

#print(data[0])

    #print(paramdf)

    radec_data = np.array([ephKBO['RA'],ephKBO['DEC']])

    dist = ourKBO.vectors(aberrations = 'astrometric')['range']
    dateList = []
    for i in jdTime:
        jd = Time(i,format='jd')
        dateList.append(jd)

    primC = SkyCoord(ra=ephKBO['RA'], dec=ephKBO['DEC'], frame='gcrs', obstime = dateList, distance = dist)
    primEcl = primC.transform_to(GeocentricTrueEcliptic(equinox='J2000'))
    
    Lat_Prim = primEcl.lat.degree
    Long_Prim = primEcl.lon.degree

#latlon = latlon_transform.convert_to_primary_centric(paramdf, ['Eris','Dysnomia'], 2, 500)
#print(latlon)

    vector = pd.DataFrame(np.array(data[0]).T,columns=['x','y','z'])
#print(vector)
#print(data[1],data[2])
    Model_df = pd.DataFrame(np.array([geo_obj_pos['time'],Lat_Prim,Long_Prim,data[1],data[2]]).T,columns=['time','Lat_prim','Lon_prim','DeltaLat','DeltaLon'])
    total_model = pd.concat([total_model, Model_df])

0


/home/byu.local/djspenc/.mm_env/lib/python3.7/site-packages/astroquery/jplhorizons/core.py:604: UserWarning: The uri used in this query is very long and might have been truncated. The results of the query might be compromised. If you queried a list of epochs, consider querying a range.
  warnings.warn(('The uri used in this query is very long '
/home/byu.local/djspenc/.mm_env/lib/python3.7/site-packages/ipykernel_launcher.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


10
20
30
40
50
60
70
80
90
100
110
120
130
140


In [78]:
print(total_model)

             time   Lat_prim   Lon_prim  DeltaLat  DeltaLon
0    2.455680e+06 -13.294767  21.823279 -0.366092  0.354709
1    2.455680e+06 -13.294772  21.824347 -0.378157  0.346813
2    2.455680e+06 -13.294777  21.825415 -0.389620  0.338365
3    2.455680e+06 -13.294792  21.826480 -0.400461  0.329375
4    2.455680e+06 -13.294806  21.827545 -0.410663  0.319860
..            ...        ...        ...       ...       ...
360  2.455716e+06 -13.320119  22.155615 -0.249224 -0.245348
361  2.455716e+06 -13.320238  22.156339 -0.232556 -0.257983
362  2.455716e+06 -13.320369  22.157069 -0.215518 -0.270208
363  2.455716e+06 -13.320488  22.157793 -0.198138 -0.282004
364  2.455716e+06 -13.320619  22.158523 -0.180441 -0.293352

[54750 rows x 5 columns]


In [74]:
epoch = runprops.get('epoch_SJD')
time_to_L2 = 5.016722408026756/24/60/60

for i in range(150):
    

    times = np.arange(2455680,2455680+36.5,0.1)
    ourKBO = Horizons(id=objname,location='399',epochs = times)
    ephKBO = ourKBO.ephemerides()['RA','DEC','datetime_jd']
    vecKBO = ourKBO.vectors(aberrations = 'astrometric')['lighttime','x','y','z']

    jdTime= ephKBO['datetime_jd']
    lightTime = vecKBO['lighttime']
    kboTime=jdTime-lightTime

    geo_obj_pos = pd.DataFrame({'kboTIME':kboTime,'x':vecKBO['x']*149597870.7 ,'y':vecKBO['y']*149597870.7 ,'z':vecKBO['z']*149597870.7 })
    geo_obj_pos['time'] = jdTime-time_to_L2
    paramdf = pd.read_csv(filename+'/'+objname+'_init_guess.csv',index_col=0)

    paramdf = paramdf.T
    paramdf['name_1'] = objname
    paramdf['name_2'] = runprops.get('names_dict').get('name_2')
    paramdf['time'] = epoch-time_to_L2
    data = np.array(mm_chisquare(paramdf, geo_obj_pos, runprops, geo_obj_pos))

#print(data[0])

    print(paramdf)

    radec_data = np.array([ephKBO['RA'],ephKBO['DEC']])

    dist = ourKBO.vectors(aberrations = 'astrometric')['range']
    dateList = []
    for i in jdTime:
        jd = Time(i,format='jd')
        dateList.append(jd)

    primC = SkyCoord(ra=ephKBO['RA'], dec=ephKBO['DEC'], frame='gcrs', obstime = dateList, distance = dist)
    primEcl = primC.transform_to(GeocentricTrueEcliptic(equinox='J2000'))
    
    Lat_Prim = primEcl.lat.degree
    Long_Prim = primEcl.lon.degree

#latlon = latlon_transform.convert_to_primary_centric(paramdf, ['Eris','Dysnomia'], 2, 500)
#print(latlon)

vector = pd.DataFrame(np.array(data[0]).T,columns=['x','y','z'])
#print(vector)
#print(data[1],data[2])
Model_df1 = pd.DataFrame(np.array([geo_obj_pos['time'],Lat_Prim,Long_Prim,data[1],data[2]]).T,columns=['time','Lat_prim','Lon_prim','DeltaLat','DeltaLon'])
print(Model_df1)

              mass_1        mass_2    sma_2   ecc_2  aop_2      inc_2  \
mean    1.620000e+22  1.600000e+20  37273.0  0.0062  170.0  61.590976   
stddev  1.000000e+21  1.000000e+19    500.0  0.0050   10.0   2.000000   

             lan_2  mea_2 name_1    name_2       time  
mean    139.117772  170.0   Eris  Dysnomia  2453979.0  
stddev    2.000000   10.0   Eris  Dysnomia  2453979.0  
          time   Lat_prim   Lon_prim  DeltaLat  DeltaLon
0    2455680.0 -13.296089  21.821579 -0.366136  0.354738
1    2455680.1 -13.296103  21.822643 -0.378201  0.346842
2    2455680.2 -13.296108  21.823712 -0.389664  0.338393
3    2455680.3 -13.296123  21.824776 -0.400506  0.329403
4    2455680.4 -13.296130  21.825822 -0.410708  0.319886
..         ...        ...        ...       ...       ...
365  2455716.5 -13.321630  22.154458 -0.162432 -0.304252
366  2455716.6 -13.321754  22.155169 -0.144190 -0.314653
367  2455716.7 -13.321875  22.155870 -0.125719 -0.324556
368  2455716.8 -13.322002  22.156590 -0.10

/home/byu.local/djspenc/.mm_env/lib/python3.7/site-packages/astroquery/jplhorizons/core.py:1048: UserWarning: The uri used in this query is very long and might have been truncated. The results of the query might be compromised. If you queried a list of epochs, consider querying a range.
  warnings.warn(('The uri used in this query is very long '
/home/byu.local/djspenc/.mm_env/lib/python3.7/site-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/byu.local/djspenc/.mm_env/lib/python3.7/site-packages/astroquery/jplhorizons/core.py:1048: UserWarning: The uri used in this query is very long and might have been truncated. The results of the query might be compromised. If you queried a list of epochs, consider querying a range.
  warnings.wa

In [ ]:
print()

In [47]:
print(Model_df-Model_df1)

    time  Lat_prim  Lon_prim      DeltaLat  DeltaLon
0    0.0 -0.000231 -0.005914  2.261508e-05 -0.000012
1    0.0 -0.000021 -0.006038  7.041991e-06  0.000010
2    0.0  0.000176 -0.005937 -3.620873e-05 -0.000009
3    0.0  0.000370 -0.005717  3.393721e-05  0.000011
4    0.0  0.000556 -0.005281  9.893141e-07 -0.000013
..   ...       ...       ...           ...       ...
68   0.0 -0.001008 -0.002672  1.473250e-05 -0.000032
69   0.0 -0.000901 -0.003624 -3.457392e-05  0.000033
70   0.0 -0.000762 -0.004377  1.980444e-05 -0.000016
71   0.0 -0.000613 -0.005072  1.231227e-05 -0.000005
72   0.0 -0.000423 -0.005565 -2.700416e-05  0.000016

[73 rows x 5 columns]


In [58]:

from astropy.coordinates import GCRS
datelist = Model_df['time']
dist = ourKBO.vectors(aberrations = 'astrometric')['range']
Model_df['Delta-RA_Secondary'] = np.zeros(len(Model_df))
Model_df['Delta-DEC_Secondary'] = np.zeros(len(Model_df))
Model_df['Delta-RA_Secondary-err'] = np.zeros(len(Model_df))
Model_df['Delta-DEC_Secondary-err'] = np.zeros(len(Model_df))

Model_df['RA_prim'] = ephKBO['RA']
Model_df['DEC_prim'] = ephKBO['DEC']
#time,Delta-RA_Secondary,Delta-RA_Secondary-err,Delta-DEC_Secondary,Delta-DEC_Secondary-err
for j in range(len(Model_df)):
    coord_sky = SkyCoord((Model_df['DeltaLat'][j]/3600+Model_df['Lat_prim'][j])*u.degree, (Model_df['DeltaLon'][j]/3600+Model_df['Lon_prim'][j])*u.degree, frame='geocentrictrueecliptic', obstime = dateList[j], distance = dist[j]*u.AU,unit=(u.deg,u.deg))
    prim_co = SkyCoord((Model_df['Lat_prim'][j])*u.degree, (Model_df['Lon_prim'][j])*u.degree, frame='geocentrictrueecliptic', obstime = dateList[j], distance = dist[j]*u.AU,unit=(u.deg,u.deg))
    
    coord_sky = coord_sky.transform_to(GCRS())
    prim_co = prim_co.transform_to(GCRS())
    
    Model_df['Delta-RA_Secondary'][j] = -(prim_co.ra.degree-coord_sky.ra.degree)*3600
    Model_df['Delta-DEC_Secondary'][j] = -(prim_co.dec.degree-coord_sky.dec.degree)*3600


/home/byu.local/djspenc/.mm_env/lib/python3.7/site-packages/astroquery/jplhorizons/core.py:1048: UserWarning: The uri used in this query is very long and might have been truncated. The results of the query might be compromised. If you queried a list of epochs, consider querying a range.
  warnings.warn(('The uri used in this query is very long '


In [79]:

from astropy.coordinates import GCRS
datelist = total_model['time']
dist = ourKBO.vectors(aberrations = 'astrometric')['range']
total_model['Delta-RA_Secondary'] = np.zeros(len(total_model))
total_model['Delta-DEC_Secondary'] = np.zeros(len(total_model))
total_model['Delta-RA_Secondary-err'] = np.zeros(len(total_model))
total_model['Delta-DEC_Secondary-err'] = np.zeros(len(total_model))

total_model['RA_prim'] = ephKBO['RA']
total_model['DEC_prim'] = ephKBO['DEC']
#time,Delta-RA_Secondary,Delta-RA_Secondary-err,Delta-DEC_Secondary,Delta-DEC_Secondary-err
for j in range(len(total_model)):
    coord_sky = SkyCoord((total_model['DeltaLat'][j]/3600+total_model['Lat_prim'][j])*u.degree, (total_model['DeltaLon'][j]/3600+total_model['Lon_prim'][j])*u.degree, frame='geocentrictrueecliptic', obstime = dateList[j], distance = dist[j]*u.AU,unit=(u.deg,u.deg))
    prim_co = SkyCoord((total_model['Lat_prim'][j])*u.degree, (Model_df['Lon_prim'][j])*u.degree, frame='geocentrictrueecliptic', obstime = dateList[j], distance = dist[j]*u.AU,unit=(u.deg,u.deg))
    
    coord_sky = coord_sky.transform_to(GCRS())
    prim_co = prim_co.transform_to(GCRS())
    
    total_model['Delta-RA_Secondary'][j] = -(prim_co.ra.degree-coord_sky.ra.degree)*3600
    total_model['Delta-DEC_Secondary'][j] = -(prim_co.dec.degree-coord_sky.dec.degree)*3600


/home/byu.local/djspenc/.mm_env/lib/python3.7/site-packages/astroquery/jplhorizons/core.py:1048: UserWarning: The uri used in this query is very long and might have been truncated. The results of the query might be compromised. If you queried a list of epochs, consider querying a range.
  warnings.warn(('The uri used in this query is very long '


ValueError: Length of values (365) does not match length of index (54750)

In [61]:
Model_df.to_csv('prep/data_points/Eris_eph.csv')

In [62]:
Model_df

,time,Lat_prim,Lon_prim,DeltaLat,DeltaLon,Delta-RA_Secondary,Delta-DEC_Secondary,Delta-RA_Secondary-err,Delta-DEC_Secondary-err,RA_prim,DEC_prim
0,2460312.0,-10.977848,23.816992,0.337071,-0.388816,0.462010,-0.228433,0.0,0.0,26.07724,-0.97418
1,2460313.0,-10.975334,23.815336,0.453796,-0.294239,0.523560,-0.098417,0.0,0.0,26.07481,-0.97243
2,2460314.0,-10.972807,23.813870,0.501111,-0.154694,0.505047,0.046620,0.0,0.0,26.07255,-0.97060
3,2460315.0,-10.970286,23.812588,0.471959,0.008417,0.409482,0.184506,0.0,0.0,26.07046,-0.96871
4,2460316.0,-10.967753,23.811497,0.370766,0.170186,0.251435,0.294177,0.0,0.0,26.06854,-0.96674
...,...,...,...,...,...,...,...,...,...,...,...
360,2460672.0,-10.792465,24.048368,-0.405827,-0.123890,-0.301559,-0.265076,0.0,0.0,26.22151,-0.71887
361,2460673.0,-10.790029,24.045707,-0.261814,-0.271859,-0.111812,-0.346036,0.0,0.0,26.21819,-0.71755
362,2460674.0,-10.787581,24.043238,-0.076738,-0.377141,0.095453,-0.372676,0.0,0.0,26.21504,-0.71615
363,2460675.0,-10.785121,24.040936,0.120269,-0.423605,0.287804,-0.341206,0.0,0.0,26.21204,-0.71468


In [75]:
1.5e9/2.99e8

5.016722408026756